## Importation des bibliothèques

In [42]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict
import tensorflow as tf
from transformers import pipeline, TFDistilBertForSequenceClassification, DistilBertTokenizer


## Création d'un document sans annotation

In [25]:
#Récupération des données annotées
tabA=pd.read_csv("edos_labelled_aggregated.csv",sep=",")

# Création d'un fichier avec les entrées non annotés
with open("non_labelled_aggregated.csv", "w", encoding="utf-8") as f_out_total:
    f_out_total.write("rewire_id,text\n")
    for i in range(len(tabA)):
        id=tabA["rewire_id"].iloc[i]
        text = tabA["text"].iloc[i]
        text_escaped = text.replace('"', '""')
        f_out_total.write(id+",\""+text_escaped+"\"\n")


In [27]:
# Regex pour les insultes sexistes spécifiques
patternInsultsSpe= r'\b(?!(?:wo)?man\b|men\b|women\b|witch[-\s]hunts?\b)(wh?[oaiy]mm?[eayi]*n?[sz]?|puss[yi]?e?s?|cunts?|\wiy?a?tche?y?s?(ing)?(tard)?|whores?(dom)?(llywood)?|slut(tiness)?s?|hoe?s?|slags?|(trad)?e?thots?(ism)?(dom)?(life)?|femoids?|harp[yi]e?s?|twats?|spinsters?|tranny|roa?sties?|stac[yi]e?s?|[vh]ags?|(noodle)?foids?|feminazis?|d[yi]kes?|skanks?|gold digg(er)?(ing)?s?|bi--th|slutt(ng)?y?)\b'
regexInsultsSpe = re.compile(patternInsultsSpe, re.IGNORECASE)

# Regex pour les termes descriptifs
patternDsc = r'\b(?!(?:man|men)\b)(she|her|\w*wom[ae]n|wi[v|f]es?|girls?|lad[y|i]e?s?|daughters?|sisters?|girlfriends?|aunti?e?s?|grandmothers?|mothers?|m[o|u]mm?[i|y]?e?s?|females?|feminine|metoo|femininity|feminis[m|t]s?|gals?|chicks?|vaginas?)\b'
regexDsc = re.compile(patternDsc, re.IGNORECASE)

# Charger le tokenizer et le modèle qui vont servir à prédire la polarité des entrées
model_name = "lxyuan/distilbert-base-multilingual-cased-sentiments-student"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name)



All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


## Filtre et classification selon la polarité

Nous avons décidé d'effectuer notre évaluation sur une quantité limitée d'entrées pour diminuer la durée de traitement. Nous avons notamment pris cette décision car notre première tentative d'évaluation sur la totalité du corpus a pris plus d'une heure.

In [36]:
with open("non_labelled_aggregated.csv", "r", encoding="utf-8") as f_in, open("données_annotees_results1000.csv", "w", encoding="utf-8") as f_out:
    f_out.write("rewire_id,text,label_sexist\n")
    for index, line in enumerate(f_in, start=1):
        if index>=2 and index<=1001:
            line=line.strip()
            matches = regexInsultsSpe.findall(line)
            if len(matches) > 0:
                f_out.write(line+",sexist\n")
            else:
                matches = re.findall(patternDsc, line)
                if len(matches) > 0:
                    inputs = tokenizer(line, return_tensors="tf", padding=True, truncation=True)
                    # Effectuer des prédictions
                    outputs = model(inputs.data)
                    # Obtenir les scores et les étiquettes prédites
                    predictions = tf.nn.softmax(outputs.logits, axis=-1)
                    predicted_labels = tf.argmax(predictions, axis=1)
                    # Définir le mapping des labels
                    label_map = {0: 'positive', 1: 'neutral', 2: 'negative'}
                    # Filtrer selon la polarité
                    for text, label in zip(line, predicted_labels):
                        label_str = label_map[label.numpy()]
                        if label_str in ['negative']:
                            f_out.write(line+",sexist\n")
                        if label_str in ['neutral','positive']:
                            f_out.write(line+",not sexist\n")
                else:
                    f_out.write(line+",not sexist\n")


## Vérification des résultats

In [46]:
tabB=pd.read_csv("données_annotees_results1000.csv",sep=",")
#On ne garde que les 1000 premières entrées pour les comparer avec les annotations
tabA_1000=tabA.iloc[0:1000]

#On vérifie que toutes les entrées ont été annotées
print("Nous avons bien les",len(tabB),"entrées annotées")

# Matrice de confusion
PRFSliste=precision_recall_fscore_support(tabA_1000["label_sexist"],tabB["label_sexist"],labels=["sexist","not sexist"])
pd.crosstab(tabA_1000["label_sexist"],tabB["label_sexist"],rownames=["Valeurs réelles"],colnames=["Valeurs prédites"])
precision=PRFSliste[0][0]
rappel=PRFSliste[1][0]
fscore=PRFSliste[2][0]
print("La précision est de "+str(precision))
print("Le rappel est de "+str(rappel))
print("Le fscore est de "+str(fscore))


Nous avons bien les 1000 entrées annotées
La précision est de 0.3926553672316384
Le rappel est de 0.8825396825396825
Le fscore est de 0.5434995112414467


Ainsi, comme nous nous en doutions d'après nos premiers résultats, nous parvenons à obtenir un assez bon rappel puisque nous avons développé notre algorithme à partir des données annotées, mais l'utilisation de la polarité n'est pas un critère assez précis pour parvenir à une bonne précision du système.